### Closures

Let's examine that concept of a cell to create an indirect reference for variables that are in multiple scopes.

In [1]:
def outer():
    x = 'python'
    def inner():
        print(x)
    return inner

In [2]:
fn = outer()

In [4]:
fn.__code__

<code object inner at 0x7f4f302b50e0, file "<ipython-input-1-438df058735b>", line 3>

In [5]:
fn.__code__.co_freevars

('x',)

As we can see, `x` is a free variable in the closure.

In [10]:
fn.__closure__

(<cell at 0x7f4f30251f70: str object at 0x7f4f339afe30>,)

Here we see that the free variable x is actually a reference to a cell object that is itself a reference to a string object.

Let's see what the memory address of `x` is in the outer function and the inner function. To be sure string interning does not play a role, I am going to use an object that we know Python will not automatically intern, like a list.

In [11]:
def outer():
    x = [1, 2, 3]
    print('outer:', hex(id(x)))
    def inner():
        print('inner:', hex(id(x)))
        print(x)
    return inner

In [19]:
fn = outer()

outer: 0x7f4f3027d200
inner: 0x7f4f3027d200
[1, 2, 3]


In [13]:
fn.__closure__

(<cell at 0x7f4f302512e0: list object at 0x7f4f3027c100>,)

In [33]:
fn()

<function __main__.outer.<locals>.inner()>

As you can see, each the memory address of `x` in `outer`, `inner` and the cell all point to the same object.

#### Modifying the Free Variable

We know we can modify nonlocal variables by using the `nonlocal` keyword. So the following will work:

In [21]:
def counter():
    count = 0 # local variable
    def inc():
        nonlocal count  # this is the count variable in counter
        count += 1
        return count
    return inc

In [22]:
c = counter()

In [23]:
c()

1

In [25]:
c()

3

In [27]:
c1 = counter()

In [28]:
c1()

1

##### Shared Extended Scopes

As we saw in the lecture, we can set up nonlocal variables in different inner functionsd that reference the same outer scope variable, i.e. we have a free variable that is shared between two closures. This works because both non local variables and the outer local variable all point back to the same cell object.

In [41]:
def outer():
    count = 0
    def inc1():
        nonlocal count
        count += 1
        return count
    
    def inc2():
        nonlocal count
        count += 1
        return count
    
    return inc1, inc2

In [42]:
fn1, fn2 = outer()

In [43]:
fn1.__closure__, fn2.__closure__

((<cell at 0x7f4f302b2c70: int object at 0x954e20>,),
 (<cell at 0x7f4f302b2c70: int object at 0x954e20>,))

As you can see here, the `count` label points to the same cell.

In [44]:
fn1()

1

In [ ]:
fn1()

In [45]:
fn2()

2

### Multiple Instances of Closures

Recall that **every** time a function is called, a **new** local scope is created.

In [47]:
from time import perf_counter

In [48]:
def func():
    x = perf_counter()
    print(x, id(x))

In [49]:
func()

28256.489734988 139977610296848


In [50]:
func()

28265.493839147 139977610296848


The same thing happens with closures, they have their own extended scope every time the closure is created:

In [51]:
def pow(n):
    # n is local to pow
    def inner(x):
        # x is local to inner
        return x ** n
    return inner

In this example, `n`, in the function `inner` is a free variable, so we have a closure that contains `inner` and the free variable `n`

In [52]:
square = pow(2)

In [53]:
square(5)

25

In [54]:
cube = pow(3)

In [55]:
cube(5)

125

We can see that the cell used for the free variable in both cases is **different**:

In [56]:
square.__closure__

(<cell at 0x7f4f13ecfe80: int object at 0x954e60>,)

In [57]:
cube.__closure__

(<cell at 0x7f4f13edafa0: int object at 0x954e80>,)

In fact, these functions (`square` and `cube`) are **not** the same functions, even though they were "created" from the same `power` function:

In [58]:
id(square), id(cube)

(139977612478064, 139977612478352)

### Beware!

Remember when I said the captured variable is a reference established when the closure is created, but the value is looked up only once the function is called?

This can create very subtle bugs in your program.

Consider the following example where we want to create some functions that can add 1, 2, 3, 4 and to whatever is passed to them.

We could do the following:

In [59]:
def adder(n):
    def inner(x):
        return x + n
    return inner

In [60]:
add_1 = adder(1)
add_2 = adder(2)
add_3 = adder(3)
add_4 = adder(4)

In [61]:
add_1(10), add_2(10), add_3(10), add_4(10)

(11, 12, 13, 14)

But suppose we want to get a little fancier and do it as follows:

In [73]:
for n in range(1, 5):
    print(n)

1
2
3
4


In [78]:
a = lambda x: x**n

In [111]:
def create_adders():
    adders = []
    for n in range(1, 5):
        adders.append(lambda x: x + n)
    return adders

In [112]:
adders = create_adders()

In [113]:
adders

[<function __main__.create_adders.<locals>.<lambda>(x)>,
 <function __main__.create_adders.<locals>.<lambda>(x)>,
 <function __main__.create_adders.<locals>.<lambda>(x)>,
 <function __main__.create_adders.<locals>.<lambda>(x)>]

Now technically we have 4 functions in the `adders` list:

In [96]:
adders

[<function __main__.create_adders.<locals>.<lambda>(x)>,
 <function __main__.create_adders.<locals>.<lambda>(x)>,
 <function __main__.create_adders.<locals>.<lambda>(x)>,
 <function __main__.create_adders.<locals>.<lambda>(x)>]

The first one should add 1 to the value we pass it, the second should add 2, and so on.

In [97]:
adders[3](10)

14

Yep, that works for the 4th function.

In [98]:
adders[0](10)

14

Uh Oh - what happened? In fact we get the same behavior from every one of those functions:

In [67]:
adders[0](10), adders[1](10), adders[2](10), adders[3](10)

(14, 14, 14, 14)

Remember what I said about when the variable is captured and when the value is looked up?

When the lambdas are **created** their `n` is the `n` used in the loop - the **same** `n`!!

In [68]:
adders[0].__code__.co_freevars

('n',)

In [69]:
adders[0].__closure__

(<cell at 0x7f4f13a7cdf0: int object at 0x954ea0>,)

In [70]:
adders[1].__closure__

(<cell at 0x7f4f13a7cdf0: int object at 0x954ea0>,)

In [71]:
adders[2].__closure__

(<cell at 0x7f4f13a7cdf0: int object at 0x954ea0>,)

In [72]:
adders[3].__closure__

(<cell at 0x7f4f13a7cdf0: int object at 0x954ea0>,)

So, by the time we call `adder[i]`, the free variable `n` (shared between all adders) is set to 4.

In [99]:
hex(id(4))

'0x954ea0'

As we can see the memory address of the singleton integer 4, is what that cell is pointint to.

If you want to use a loop to do this and not end up using the same cell for each of the free variables, we can use a simple trick that forces the evaluation of `n` at the time the closure is **created**, instead of when the closure function is evaluated.

We can do this by creating a parameter for `n` in our lambda whose default value is the current value of `n` - remember from an earlier video that parameter defaults are avaluated when the function is created, not called.

In [ ]:
def outer():
    n=0 
    def inner():
        print(n)
    return inner

In [114]:
def create_adders():
    adders = []
    for n in range(1, 5):
        adders.append(lambda x, step=n: x + step)
    return adders

In [115]:
adders = create_adders()

In [116]:
adders

[<function __main__.create_adders.<locals>.<lambda>(x, step=1)>,
 <function __main__.create_adders.<locals>.<lambda>(x, step=2)>,
 <function __main__.create_adders.<locals>.<lambda>(x, step=3)>,
 <function __main__.create_adders.<locals>.<lambda>(x, step=4)>]

In [117]:
adders[0].__closure__

Why aren't we getting anything in the closure? What about free variables?

In [118]:
adders[0].__code__.co_freevars

()

Hmm, nothing either... Why?

Well, look at the lambda in that loop. Does it reference the variable `n` (other than in the default value)? No. Hence, `n` is **not** a free variable in this case, and our lambda is just a plain lambda, not a closure.

And this code will now work as expected:

In [107]:
adders[0](10)

11

In [108]:
adders[1](10)

12

In [109]:
adders[2](10)

13

In [ ]:
adders[3](10)

You just need to understand that since the default values are evaluated when the function (lambda in this case) is **created**, the then-current `n` value is assigned to the local variable `step`. So `step` will not change every time the lambda is called, and since n is not referenced inside the function (and therefore evaluated when the lambda is called), `n` is not a free variable.

#### Nested Closures

We can also nest closures, as can be seen in this example:

In [120]:
def incrementer(n):
    def inner(start):
        current = start
        def inc():
            a = 10  # local var
            nonlocal current
            current += n
            return current
        return inc
    return inner
        

In [121]:
fn = incrementer(2)

In [122]:
fn

<function __main__.incrementer.<locals>.inner(start)>

In [123]:
fn.__code__.co_freevars

('n',)

In [124]:
fn.__closure__

(<cell at 0x7f4f2823b4f0: int object at 0x954e60>,)

In [125]:
inc_2 = fn(100)

In [126]:
inc_2

<function __main__.incrementer.<locals>.inner.<locals>.inc()>

In [127]:
inc_2.__code__.co_freevars

('current', 'n')

In [128]:
inc_2.__closure__

(<cell at 0x7f4f13ecfdf0: int object at 0x955aa0>,
 <cell at 0x7f4f2823b4f0: int object at 0x954e60>)

Here you can see that the second free variable `n`, is pointing to the same cell as the free variable in `fn`.

Note that **a** is a local variable, and is not considered a free variable.

And we can call the closures as follows:

In [129]:
inc_2()

102

In [130]:
inc_2()

104

In [131]:
inc_3 = incrementer(3)(200)

In [132]:
inc_3()

203

In [133]:
inc_3()

206